### Exercises

#### Question 1

Alongside this notebook is a data file named `daily_quotes.csv` which contains EOD OHLC/Volume data for a small number of equities over a 6 month period.

The first step is to load up this data into a dataframe, ensuring that all data types are correct (datetime objects for dates, floats for OHLC data, and integers for Volume).

Write a function that receives the file name as an argument and returns a dataframe that:
- has the correct data type for each column (`str`, `float`, `int`)
- has a row index based on the `symbol` column

In addition, we would like our dataframe to contain columns named and ordered in a specific way:
- symbol (`str`)
- date (`datetime`)
- open (`float`)
- high (`float`)
- low (`float`)
- close (`float`)
- volume (`int`)

(with `symbol` being used as the row index)

Hint: 

You will want to read up the Pandas docs for `read_csv` to see how you can handle datetime data directly while loading the data (in particular you should look at the `parse_dates` option):

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

Alternatively, you could convert these objects into proper datetime types after loading by using the Pandas function `to_datetime`, documented here:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

and then use conatenation to build up a dataframe that replaces the "old" `date` column with the "new" (properly typed) column.

### solution

In [54]:
import pandas as pd

def load_data(file_name):
    # Read the CSV file and specify the data types
    df = pd.read_csv(file_name, parse_dates=['Date'], dtype={'Symbol': str, 'Low': float, 'Close/Last': float, 'Volume': int})
    
    df.set_index('Symbol', inplace=True)
    
    # Reorder the columns
    df = df[['Date', ' Close/Last', ' Volume', ' Open', ' High', ' Low']]
    
    return df


In [55]:
file_name = 'daily_quotes.csv'
df = load_data(file_name)
df.head()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5132\483800711.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_name, parse_dates=['Date'], dtype={'Symbol': str, 'Low': float, 'Close/Last': float, 'Volume': int})


,Date,Close/Last,Volume,Open,High,Low
Symbol,,,,,,
AAPL,2021-02-12,135.37,60145130,134.350,135.53,133.6921
AMZN,2021-02-12,3277.71,2335339,3250.000,3280.25,3233.3100
GOOG,2021-02-12,2104.11,855865,2090.250,2108.82,2083.1300
MSFT,2021-02-12,244.99,16561080,243.933,245.30,242.7300
AAPL,2021-02-11,135.13,64280030,135.900,136.39,133.7700


#### Question 2

Write a function that, given a dataframe sructured as the one we created in Question 1 and a symbol name as a string (e.g. `AAPL`, `MSFT`, etc), will:
- return a similarly structured dataframe consisting of the row (or rows) containing the records with the highest volume for the given symbol
- raises a `ValueError` if the symbol is not in the dataframe

#### Solution

In [60]:
def get_highest_volume(df, symbol):
    symbol_df = df[df.index == symbol]

    if symbol_df.empty:
        raise ValueError(f"Symbol '{symbol}' not found in the dataframe.")
    
    max_volume = symbol_df[' Volume'].max()
   
    result_df = symbol_df[symbol_df[' Volume'] == max_volume]
    
    return result_df

In [61]:
highest_volume_df = get_highest_volume(df, 'MSFT')
highest_volume_df


,Date,Close/Last,Volume,Open,High,Low
Symbol,,,,,,
MSFT,1/27/21,232.9,69870640,238.0,240.44,230.14


#### Question 3

Using the same dataframe as in the preceding questions, our goal now is to write a function that will return, for a specific symbol, the row that had the largest high-low spread.

Write a function to do that - it should just return a dataframe with the row (or rows) with the largest high-low spread.

### Solution

In [64]:
df.columns

Index(['Date', ' Close/Last', ' Volume', ' Open', ' High', ' Low'], dtype='object')

In [65]:
def get_largest_spread(df, symbol):
    symbol_df = df[df.index == symbol]

    if symbol_df.empty:
        raise ValueError(f"Symbol '{symbol}' not found in the dataframe.")
    
    # Calculate the high-low spread
    symbol_df['Spread'] = symbol_df[' High'] - symbol_df[' Low']
    
    # Find the maximum spread
    max_spread = symbol_df['Spread'].max()
    
    # Filter the dataframe for rows with the maximum spread
    result_df = symbol_df[symbol_df['Spread'] == max_spread]
    
    return result_df


In [66]:
largest_spread=get_largest_spread(df, 'AAPL')
largest_spread

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_5132\20137581.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  symbol_df['Spread'] = symbol_df[' High'] - symbol_df[' Low']


,Date,Close/Last,Volume,Open,High,Low,Spread
Symbol,,,,,,,
AAPL,9/4/20,120.96,332607200,120.07,123.7,110.89,12.81


#### Question 4

Using the same dataframe as the preceding questions, write a function that returns a single dataframe containing the record(s) with maximum high-low spread for each symbol in the dataframe. (Do not hardcode symbol names in this function - instead you should recover the possible symbol names from the data itself).

The returned dataframe should have the same structure as the original dataframe, but just contain the rows of maximum high-low spread for each symbol.

### solutions

In [71]:
import pandas as pd

def get_max_spread_per_symbol(df):
    df['Spread'] = df[' High'] - df[' Low']
    
    max_spreads = df.groupby(df.index)['Spread'].max()
    
    result_df = pd.DataFrame(columns=df.columns)
   
    for symbol, max_spread in max_spreads.items():
        symbol_df = df[df.index == symbol]
        max_spread_rows = symbol_df[symbol_df['Spread'] == max_spread]
        result_df = pd.concat([result_df, max_spread_rows])
    
    return result_df


In [72]:
max_spread_per_symbol=get_max_spread_per_symbol(df)
max_spread_per_symbol

,Date,Close/Last,Volume,Open,High,Low,Spread
AAPL,9/4/20,120.96,332607200,120.07,123.70,110.8900,12.8100
AMZN,9/4/20,3294.62,8781754,3318.00,3381.50,3111.1300,270.3700
GOOG,2/3/21,2070.07,4118170,2073.00,2116.50,2018.3800,98.1200
MSFT,9/3/20,217.30,58400290,229.27,229.31,214.9602,14.3498
